In [4]:
import os
from tika import parser
import re
import pandas as pd
from bs4 import BeautifulSoup

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


path = os.getcwd()


In [53]:
path = r"C:\Users\luiz\Desktop\flavio_novo"
path=path+"\\"

In [4]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
for pdf in folders:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
        #print("ALERTA",pdf)
    folhas = re.findall(r"Data pregão(.*?)XP",a.replace('\n',''))
    aux = '_'.join(folhas[0].split('/')[::-1])
    #print(aux)
    os.rename(path+pdf,path+aux+".pdf")

NameError: name 'os' is not defined

In [55]:
list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))

['2614495_NOTABMF_01012020.pdf',
 '2614495_NOTABMF_01022020.pdf',
 '2614495_NOTABMF_01032020.pdf',
 '2614495_NOTABMF_01042020.pdf',
 '2614495_NOTABMF_01062020.pdf',
 '2614495_NOTABOV_01012020.pdf',
 '2614495_NOTABOV_01022020.pdf',
 '2614495_NOTABOV_01032020.pdf',
 '2614495_NOTABOV_01042020.pdf',
 '2614495_NOTABOV_01052020.pdf',
 '2614495_NOTABOV_01062020.pdf',
 '2614495_NOTABOV_01102019.pdf',
 '2614495_NOTABOV_01112019.pdf',
 '2614495_NOTABOV_01122019.pdf',
 '2614495_NotaCorretagem.pdf']

In [68]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(r"C:\Users\luiz\Desktop\flavio\nota_2\vendas\\")))
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
#for pdf in folders:
for pdf in ['2614495_NOTABOV_01102019.pdf']:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    #########
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
    if 'WIN' in a:
        print("ALERTA WIN",pdf)
    if 'IND '  in a:
        print("ALERTA IND",pdf)
    ##########
    folhas = re.findall(r"NOTA DE CORRETAGEMNr. nota(.*?)Folha(.*?)Data pregão(.*?)XP",a.replace('\n',''))

    #taxas = re.findall(r"Precatório\n([(0-9|.|,)]*)Total CBLC (D|C)\n([(0-9|.|,)]*)Valor líquido das operações (D|C)\n([(0-9|.|,)]*)Taxa de liquidação (D|C)\n([(0-9|.|,)]*)Taxa de Registro (D|C)\n([(0-9|.|,)]*)Total Bovespa / Soma (D|C)\n([(0-9|.|,)]*)Taxa de termo/opções (D|C)\n([(0-9|.|,)]*)Taxa A.N.A. (D|C)\n([(0-9|.|,)]*)Emolumentos (D|C)\n \n([(0-9|.|,)]*)Total corretagem / Despesas (D|C)\n \n([(0-9|.|,)]*)Taxa Operacional (D|C)\n([(0-9|.|,)]*)Execução\n([(0-9|.|,)]*)Taxa de Custódia\n([(0-9|.|,)]*)Impostos (D|C)\n([(0-9|.|,)]*)I.R.R.F. s/ operações, base R\$(.*?)\n([(0-9|.|,)]*)Outros (C|D)\n([(0-9|.|,)]*)Líquido para (.*?) (C|D)",a)
    taxas= re.findall(r"Precatório\n([(0-9|.|,)]*)Total CBLC (D|C)\n([(0-9|.|,)]*)Valor líquido das operações (D|C)\n([(0-9|.|,)]*)Taxa de liquidação (D|C)\n([(0-9|.|,)]*)Taxa de Registro (D|C)\n([(0-9|.|,)]*)Total Bovespa / Soma (D|C)\n([(0-9|.|,)]*)Taxa de termo/opções (D|C)\n([(0-9|.|,)]*)Taxa A.N.A. (D|C)\n([(0-9|.|,)]*)Emolumentos (D|C)\n \n([(0-9|.|,)]*)Total corretagem / Despesas (D|C)\n \n([(0-9|.|,)]*)Clearing (D|C)\n([(0-9|.|,)]*)Execução\n([(0-9|.|,)]*)Execução casa\n([(0-9|.|,)]*)ISS \(SÃO PAULO\) (D|C)\n([(0-9|.|,)]*)I.R.R.F. s/ operações, base R\$(.*?)\n([(0-9|.|,)]*)Outras Bovespa (C|D)\n([(0-9|.|,)]*)Líquido para (.*?) (C|D)",a)
    #print(taxas)
    for i in range(len(taxas)):
        taxas[i] = list(taxas[i])
        if taxas[i][-6][-1] != "D":
            taxas[i][-7] = '0,00'
        assert taxas[i][20] == taxas[i][20] =='0,00'
        for j in range(15):
            if taxas[i][2*j+1][-1] == 'D':
                taxas[i][2*j] = '-'+taxas[i][2*j]
        if taxas[i][-1] == 'D':
                taxas[i][-3] = '-'+taxas[i][-3]
        taxas[i] = list(map(lambda x : int(x.replace('.','').replace(',','')) if not '/' in x else '-'.join(x.split('/')[::-1]),taxas[i][:-1:2] +[taxas[i][-2]]+[taxas[i][-6].split(' ')[0]]))
        try:
            lucro_daytrade,IRRF_daytrade = re.findall(r"IRRF Day Trade: Base R\$ (.*?) Projeção R\$ (.*?)\n",a)[0]
            taxas[i].append(-int(lucro_daytrade.replace('.','').replace(',','')))
            taxas[i].append(-int(IRRF_daytrade.replace('.','').replace(',','')))
        except:
            taxas[i].append(0)
            taxas[i].append(0)
        assert taxas[i][0] == sum(taxas[i][1:4])
        assert taxas[i][4] == sum(taxas[i][5:8])
        assert taxas[i][8] == sum(taxas[i][9:14])
        assert taxas[i][0] + taxas[i][4]+ taxas[i][8] + taxas[i][-1]== taxas[i][-5]
    #for k in taxas:
    #    print(k[-1],k[-5])
    #    print((k[1]*(1+0.00004105+0.0002795)+k[9]*(0.1068+0.039+1))/100 - k[-3]/100)

    trades = re.findall(r"Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/C(.*?)NOTA DE CORRETAGEM",a.replace('\n',''))
    #trades2 = re.findall(r"CONTINUA...Corretagem / DespesasLíquido paraObservação: \(1\) As operações a termo não são computadas no líquido da fatura.Capitais e regiões metropolitanas: 4003-3710 Demais localidades: 0800-880-3710 SAC: 0800-772-0202 Ouvidoria: 0800-722-3710(.*?)Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/CNOTA DE CORRETAGEM",a.replace('\n',''))
    trades2 = re.findall(r"CONTINUA...Corretagem / DespesasLíquido paraObservação: \(1\) As operações a termo não são computadas no líquido da fatura.(.*?)Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/CNOTA DE CORRETAGEM",a.replace('\n',''))
    assert trades.count('') == len(trades2)
    for i,j in enumerate(folhas):
        if j[1] != '1':
            assert trades[i] == ''
            trades[i] = trades2[0]
            trades2 = trades2[1:]


        aux = filter(lambda x: x !='',trades[i].split('1-'))
        aux2 = map(lambda x : re.findall(r"^(.*?) (C|V) (.*?) (.*?) ([(0-9|.|,)]*) ([(0-9|.|,)]*) ([(0-9|.|,)]*) (D|C)$",x)[0],aux)
        aux3 = map(lambda x : [x[0],x[1],x[2],re.sub(' +', ' ', x[3]).strip(),x[4].replace('.','').replace(',','.'),x[5].replace('.','').replace(',','.'),x[6].replace('.','').replace(',','.'),x[7]],aux2)
        folhas[i] = [j[0],j[1],'-'.join(j[2].split('/')[::-1]),list(aux3)] 

    assert len(trades2)==0

    folhas = folhas[::-1]
    for i,k in enumerate(folhas):
        #Aqui to perdendo a info do marcado D do daytrade no nome do ativo
        k[3] = list(map(lambda x : (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int('-'+x[4]),x[5],int(x[6].replace('.',''))) if x[1] == 'V' else (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int(x[4]),x[5],int('-'+x[6].replace('.',''))) ,k[3]))
        if i != 0:
            if folhas[i-1][1] != '1':
                folhas[i][3]= folhas[i][3]+folhas[i-1][3]
    folhas = list(filter(lambda x: x[1] == '1',folhas[::-1]))
    folhas = list(map(lambda x: x[3],folhas))
    assert len(folhas)==len(taxas)

    historico.append([n for m in folhas for n in m])
    taxa.append(taxas)
historico,taxa = [n for m in historico for n in m],[z for x in taxa for z in x]


In [70]:

def translate_tickers(nome):
    tickers={
        'BTGP BANCO UNT N2':'BPAC11',
        'BTGP BANCO UNT N2 D':'BPAC11',
        'ENEVA ON NM':'ENEV3',
        'ENEVA ON NM D':'ENEV3',
        'BIOTOSCANA DR3':'GBIO33',
        'BIOTOSCANA DR3 D#2':'GBIO33',
        'BIOTOSCANA DR3 D':'GBIO33',
        'QUALICORP ON ER NM':'QUAL3',
        'QUALICORP ON NM':'QUAL3',
        'BRASIL ON NM #2D':'BBAS3',
        'BRASIL ON NM':'BBAS3',
        'BRASIL ON NM D':'BBAS3',
        'SINQIA ON NM':'SQIA3',
        'AES TIETE E UNT ED N2':'TIET11',
        'AES TIETE E UNT N2':'TIET11',
        'BRADESCO PN EJB N1':'BBDC4',
        'BRADESCO PN N1':'BBDC4',
        'BRADESCO ON EJB N1':'BBDC3',
        'BRADESCO ON N1':'BBDC3',
        'ITAUUNIBANCO PN ED N1':'ITUB4',
        'ITAUUNIBANCO PN N1':'ITUB4',
        'JBS ON ED NM':'JBSS3',
        'JBS ON NM':'JBSS3',
        'LOJAS RENNER ON EDB NM':'LREN3',
        'PETROBRAS PN N2 D':'PETR4',
        'LOJAS RENNER ON NM':'LREN3',
        'P.ACUCAR-CBD PN EJ N1':'PCAR4',
        'P.ACUCAR-CBD PN N1':'PCAR4',
        'PETROBRAS PN ED N2':'PETR4',
        'PETROBRAS PN N2':'PETR4',
        'BRASIL ON NM D':'BBAS3',
        'VALE ON NM':'VALE3',
        'AZUL PN N2':'AZUL4',
        'OI ON N1':'OIBR3',
        'OI ON N1 D':'OIBR3',
        'OI ON N1 D#2':'OIBR3',
        'B2W DIGITAL ON NM':'BTOW3',
        'ENGIE BRASIL ON NM':'EGIE3',
        'EQUATORIAL ON NM':'EQTL3',
        'LOCALIZA ON NM':'RENT3',
        'SUZANO PAPEL ON NM':'SUZB3',
        'CARREFOUR BR ON NM':'CRFB3',
        'TIM PART S/A ON NM':'TIMP3',
        'SUL AMERICA UNT N2':'SULA11',
        'MULTIPLAN ON N2':'MULT3',
        'TRAN PAULIST PN N1':'TRPL4',
        'RUMO S.A. ON NM':'RAIL3',
        'LOJAS RENNER ON ED NM D':'LREN3',
        'EZTEC ON NM':'EZTC3',
        'EZTEC ON EDB NM':'EZTC3',
        'B3 ON NM':'B3SA3',
        'ISHARES BOVA CI ATZ':'BOVA11',
        'LOCAWEB ON NM':'LWSA3',
        'RANDON PART PN N1':'RAPT4',
        'IGUATEMI ON NM' : 'IGTA3',
        'BR MALLS PAR ON NM':'BRML3',
        'MAGAZ LUIZA ON NM D':'MGLU3',
        'CVC BRASIL ON NM':'CVCB3',
        'TOTVS ON NM':'TOTS3',
        'KEPLER WEBER ON EJ':'KEPL3',
        'KEPLER WEBER ON':'KEPL3',
        'TUPY ON NM':'TUPY3',
        'PETRORIO ON NM':'PRIO3',
        'PETRORIO ON NM D':'PRIO3',
        'PETRORIO ON NM D#2':'PRIO3',
        'VIAVAREJO ON NM':'VVAR3',
        'CENTAURO ON NM':'CNTO3',
        'ELETROBRAS ON N1':'ELET3',
        'BRADESCO PN EJ N1':'BBDC4',
        'M.DIASBRANCO ON EJ NM':'MDIA3',
        'ABC BRASIL PN EJ N2':'ABCB4',
        'CAMIL ON NM':'CAML3',
        'GRUPO NATURA ON NM':'NTCO3',
        'CEA MODAS ON EJ NM':'CEAB3',
        'CEA MODAS ON NM':'CEAB3',
        'CIA HERING ON NM':'HGTX3',
        'IMC S/A ON NM':'MEAL3',
        'PPLA UNT':'PPLA11',
        'BANCO BMG PN N1':'BMGB4',
        'LOG-IN ON NM':'LOGN3',
        'DE COMPRA 01/20 VVARA125 ON 12,50 VVAR':'VVAR125',
        'OPC COMPRA 01/20 VVARA125E ON 12,50':'VVAR125',
        'PORTOBELLO ON NM':'PTBL3',
        'RECRUSUL PN':'RCSL4',
        'GAFISA ON NM':'GFSA3',
        'COGNA ON ON NM':'COGN3',
        'MILLS ON NM':'MILS3',
        'TECNISA ON NM':'TCSA3',
        'TRISUL ON NM' : 'TRIS3',
        'CEMIG PN N1' : 'CMIG4',
        'CEMIG PN EJ N1': 'CMIG4',
        'DOMMO ON':'DMMO3',
        'PETROBRAS BR ON NM':'BRDT3',
        'OI PN N1':'OIBR4',
        'ISHARES BOVA CI':'BOVA11',
        'ISHARES BOVA CI D':'BOVA11',
        'ELETROBRAS PNB N1':'ELET6',
        'FII XP CRED XPCI12 DM 100,51':'DSHFOSDHFOISDFJSDPOOFJSPOFJSDPF',
    }
    return tickers[nome]


In [71]:
aux = pd.DataFrame(historico,columns=['nota','pregao','nome','qtd','preco_medio','financeiro'])
aux['daytrade'] = aux.nome.apply(lambda x : True if x[-1] == 'D' or x[-3:-1] == 'D#' or x[-4:-1] == 'D#2' else False)
aux.nome = aux.nome.apply(translate_tickers)
aux['mes'] = aux['pregao'].apply(lambda x : '-'.join(x.split('-')[:2]))
aux = aux.sort_values(by='pregao',ascending=True)
aux.index = range(len(aux))
aux['turnover'] = abs(aux['financeiro'])



In [72]:
aux

,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
0,16956305,2019-10-17,BBAS3,1000,44.93,-4493000,False,2019-10,4493000
1,16956305,2019-10-17,ENEV3,1900,29.61,-5625900,False,2019-10,5625900
2,16977078,2019-10-18,RAPT4,100,11.00,-110000,False,2019-10,110000
3,16977078,2019-10-18,RAPT4,2200,11.00,-2420000,False,2019-10,2420000
4,16977078,2019-10-18,RAPT4,4800,11.00,-5280000,False,2019-10,5280000
...,...,...,...,...,...,...,...,...,...
105,17185052,2019-10-31,BRML3,400,15.36,-614400,False,2019-10,614400
106,17185052,2019-10-31,BRML3,200,15.36,-307200,False,2019-10,307200
107,17185052,2019-10-31,BRML3,400,15.36,-614400,False,2019-10,614400
108,17185052,2019-10-31,IGTA3,400,48.00,-1920000,False,2019-10,1920000


In [73]:
turnover = aux.groupby(by=['pregao','nota']).sum()[["turnover"]].sort_values(by='pregao',ascending=True)
turnover

,,turnover
pregao,nota,
2019-10-17,16956305,10118900
2019-10-18,16977078,21628200
2019-10-21,16998228,1719500
2019-10-22,17022090,8252200
2019-10-23,17045946,11977500
2019-10-24,17076198,151354000
2019-10-30,17161639,10482400
2019-10-31,17185052,7679600


In [74]:
aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)


,pregao,nota
0,2019-10-17,16956305
1,2019-10-18,16977078
2,2019-10-21,16998228
3,2019-10-22,17022090
4,2019-10-23,17045946
5,2019-10-24,17076198
6,2019-10-30,17161639
7,2019-10-31,17185052


In [75]:
df_taxas = pd.DataFrame(taxa).sort_values(by=15,ascending=True)
df_taxas.columns = ['CBLC','valor_liquido_operacoes','taxa_liquidacao','taxa_registro','total_bovespa','taxa_termo_opcoes','taxa_ANA','emolumentos','total_custos','taxa_operacional','execucao','impostos','IRRF','outros','valor_liquido_para','data_liquidacao','montante_IRRF','lucro_daytrade','IRRF_daytrade']
df_taxas.index = range(len(df_taxas))
#df_taxas = df_taxas[[15,14]]
#df_taxas.columns = ['liquidacao','IRRF','valorIRRF','corretagem','valor_final','valor_liquid_operacoes','lucro_daytrade','IRRF_daytrade']
#df_taxas['valor_liquido_para'] = df_taxas['valor_liquido_para'] - df_taxas['IRRF_daytrade']
df_taxas['custos_operacionais'] = df_taxas['taxa_liquidacao']+df_taxas['emolumentos']+df_taxas['impostos']+df_taxas['outros']+df_taxas['taxa_operacional']
df_taxas = pd.concat([df_taxas,aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)],axis=1)
#df_taxas= df_taxas[['nota','pregao','data_liquidacao','taxa_operacional','custos_operacionais','valor_liquido_para']]
#df_taxas.set_index(15).T
df_taxas

,CBLC,valor_liquido_operacoes,taxa_liquidacao,taxa_registro,total_bovespa,taxa_termo_opcoes,taxa_ANA,emolumentos,total_custos,taxa_operacional,...,IRRF,outros,valor_liquido_para,data_liquidacao,montante_IRRF,lucro_daytrade,IRRF_daytrade,custos_operacionais,pregao,nota
0,-10121682,-10118900,-2782,0,-578,0,0,-578,-4330,-3780,...,0,-147,-10126590,2019-10-21,0,0,0,-7690,2019-10-17,16956305
1,-12016147,-12010200,-5947,0,-887,0,0,-887,-11067,-9450,...,-240,-368,-12028101,2019-10-22,4809000,0,0,-17661,2019-10-18,16977078
2,1719028,1719500,-472,0,-70,0,0,-70,-2164,-1890,...,0,-73,1716794,2019-10-23,1719500,0,0,-2706,2019-10-21,16998228
3,8249931,8252200,-2269,0,-338,0,0,-338,-2576,-1890,...,-412,-73,8247017,2019-10-24,8252200,0,0,-4771,2019-10-22,17022090
4,11974207,11977500,-3293,0,-491,0,0,-491,-9259,-7560,...,-598,-294,11964457,2019-10-25,11977500,0,0,-12445,2019-10-23,17045946
5,137444778,137486400,-41622,0,-6213,0,0,-6213,-22378,-13230,...,-7221,-515,137416187,2019-10-28,144420200,0,0,-62992,2019-10-24,17076198
6,-10485282,-10482400,-2882,0,-540,0,0,-540,-6496,-5670,...,0,-221,-10492318,2019-11-01,0,0,0,-9918,2019-10-30,17161639
7,-7681711,-7679600,-2111,0,-315,0,0,-315,-4330,-3780,...,0,-147,-7686356,2019-11-04,0,0,0,-6756,2019-10-31,17185052


In [78]:
#verifica se os daytrades estão todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

,,qtd,financeiro,daytrade,turnover
pregao,nome,,,,


In [59]:
#preciso na unha tirar os BOVA11 q na vdd n foram daytrade
confusao = aux[aux.daytrade][aux.nome == 'BOVA11']
confusao

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
860,22678282,2020-06-15,BOVA11,-1000,88.99,8899000,True,2020-06,8899000
862,22678282,2020-06-15,BOVA11,-10,88.98,88980,True,2020-06,88980
863,22678282,2020-06-15,BOVA11,-70,88.99,622930,True,2020-06,622930
864,22678282,2020-06-15,BOVA11,-1190,88.99,10589810,True,2020-06,10589810
865,22678282,2020-06-15,BOVA11,-400,88.98,3559200,True,2020-06,3559200
866,22678282,2020-06-15,BOVA11,-10,88.99,88990,True,2020-06,88990
867,22678282,2020-06-15,BOVA11,-13100,88.98,116563800,True,2020-06,116563800
868,22678282,2020-06-15,BOVA11,7680,88.97,-68328960,True,2020-06,68328960
869,22678282,2020-06-15,BOVA11,200,88.96,-1779200,True,2020-06,1779200
870,22678282,2020-06-15,BOVA11,10,88.97,-88970,True,2020-06,88970


In [60]:
qtd_comprada = confusao[confusao.qtd>0].qtd.sum()
qtd_vendida  = -confusao[confusao.qtd<0].qtd.sum()

#saldo final dos trades daquele dia, se ficou net comprado ou net vendido
saldo = qtd_comprada-qtd_vendida

#medio de compra
medio_compra = -0.01*confusao[confusao.qtd>0].financeiro.sum()/qtd_comprada
#medio_venda
medio_venda = 0.01*confusao[confusao.qtd<0].financeiro.sum()/qtd_vendida
medio_compra,medio_venda


(88.96974651457542, 88.98143852978454)

In [61]:
adicionar = confusao.iloc[0:2].copy()
adicionar.index = [0,1]
adicionar

,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
0,22678282,2020-06-15,BOVA11,-1000,88.99,8899000,True,2020-06,8899000
1,22678282,2020-06-15,BOVA11,-10,88.98,88980,True,2020-06,88980


In [62]:

if saldo > 0 and medio_compra < medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra < medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

elif saldo > 0 and medio_compra > medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra > medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

    
else:
    print('algum erro de caso inesperado')


#adicionar.index = [len(aux),len(aux)+1]
adicionar['turnover']  = adicionar['financeiro'].apply(abs)

In [63]:
aux.drop(confusao.index,inplace=True)
aux = pd.concat([aux,adicionar]).sort_values('pregao')

In [79]:
#agora todos os daytrades devem estar todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

,,qtd,financeiro,daytrade,turnover
pregao,nome,,,,


In [80]:
#Quantidade de dinheiro ganhado no daytrade BVM (SEM INDICIE FUTURO) (ainda não estao incluidos os custos)
e['financeiro'].sum()/100

0.0

In [81]:
taxas_detalhadas = df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]
taxas_detalhadas

,nota,pregao,data_liquidacao,valor_liquido_operacoes,custos_operacionais,valor_liquido_para,IRRF,IRRF_daytrade
0,16956305,2019-10-17,2019-10-21,-10118900,-7690,-10126590,0,0
1,16977078,2019-10-18,2019-10-22,-12010200,-17661,-12028101,-240,0
2,16998228,2019-10-21,2019-10-23,1719500,-2706,1716794,0,0
3,17022090,2019-10-22,2019-10-24,8252200,-4771,8247017,-412,0
4,17045946,2019-10-23,2019-10-25,11977500,-12445,11964457,-598,0
5,17076198,2019-10-24,2019-10-28,137486400,-62992,137416187,-7221,0
6,17161639,2019-10-30,2019-11-01,-10482400,-9918,-10492318,0,0
7,17185052,2019-10-31,2019-11-04,-7679600,-6756,-7686356,0,0


In [82]:
assert (df_taxas.valor_liquido_operacoes+df_taxas.custos_operacionais+df_taxas.IRRF+df_taxas.IRRF_daytrade == df_taxas.valor_liquido_para).all()
assert (aux.groupby(['nota']).sum().financeiro.values == taxas_detalhadas.valor_liquido_operacoes.values).all()

In [83]:
df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]

,nota,pregao,data_liquidacao,valor_liquido_operacoes,custos_operacionais,valor_liquido_para,IRRF,IRRF_daytrade
0,16956305,2019-10-17,2019-10-21,-10118900,-7690,-10126590,0,0
1,16977078,2019-10-18,2019-10-22,-12010200,-17661,-12028101,-240,0
2,16998228,2019-10-21,2019-10-23,1719500,-2706,1716794,0,0
3,17022090,2019-10-22,2019-10-24,8252200,-4771,8247017,-412,0
4,17045946,2019-10-23,2019-10-25,11977500,-12445,11964457,-598,0
5,17076198,2019-10-24,2019-10-28,137486400,-62992,137416187,-7221,0
6,17161639,2019-10-30,2019-11-01,-10482400,-9918,-10492318,0,0
7,17185052,2019-10-31,2019-11-04,-7679600,-6756,-7686356,0,0


In [84]:
df_taxas.IRRF.sum()/100,df_taxas.IRRF_daytrade.sum()/100,taxas_detalhadas[['IRRF','IRRF_daytrade']].sum().sum()/100

(-84.71, 0.0, -84.71)

In [85]:
teste = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
teste

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover
0,16956305,2019-10-17,2019-10,BBAS3,False,1000,-4493000,4493000
1,16956305,2019-10-17,2019-10,ENEV3,False,1900,-5625900,5625900
2,16977078,2019-10-18,2019-10,BBAS3,False,400,-1859200,1859200
3,16977078,2019-10-18,2019-10,ITUB4,False,-1400,4809000,4809000
4,16977078,2019-10-18,2019-10,QUAL3,False,2000,-6710000,6710000
5,16977078,2019-10-18,2019-10,RAPT4,False,7500,-8250000,8250000
6,16998228,2019-10-21,2019-10,ITUB4,False,-500,1719500,1719500
7,17022090,2019-10-22,2019-10,RAPT4,False,-7500,8252200,8252200
8,17045946,2019-10-23,2019-10,ENEV3,False,-1900,5787500,5787500
9,17045946,2019-10-23,2019-10,QUAL3,False,-2000,6190000,6190000


In [87]:
#valor em REAIS que foi cobrado a mais no 'taxa_liquidacoo'
print((-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795).sum()/100)
(-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795)/100

-10.098378500000003


0   -0.462326
1   -0.980819
2   -0.086003
3   -0.374899
4   -0.547113
5   -6.814430
6   -0.478308
7   -0.354482
dtype: float64

In [88]:
#valor em REAIS que foi cobrado a mais no 'emolumento'
print((-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105).sum()/100)
(-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105)/100

2.691350849999992


0    1.626192
1   -0.008376
2   -0.005855
3   -0.007528
4   -0.006764
5   -0.000817
6    1.096975
7   -0.002476
dtype: float64

In [89]:
#valor em centavos que foi cobrado a mais no 'imposto'
-df_taxas.impostos+(0.1068)*df_taxas.reset_index()['taxa_operacional']

0   -0.704
1   -0.260
2   -0.852
3   -0.852
4   -0.408
5   -0.964
6   -0.556
7   -0.704
dtype: float64

In [90]:
#valor em centavos que foi cobrado a mais na 'outros'
-df_taxas.outros+(0.039)*df_taxas.reset_index()['taxa_operacional']

0   -0.42
1   -0.55
2   -0.71
3   -0.71
4   -0.84
5   -0.97
6   -0.13
7   -0.42
dtype: float64

In [91]:
#taxa_liquidacoo
-turnover.reset_index().turnover*0.0002795/100
#emolumentos
-turnover.reset_index().turnover*0.00004105/100
#impostos
(0.1068)*df_taxas.reset_index()['taxa_operacional']/100
#outros
(0.039)*df_taxas.reset_index()['taxa_operacional']/100

0   -1.4742
1   -3.6855
2   -0.7371
3   -0.7371
4   -2.9484
5   -5.1597
6   -2.2113
7   -1.4742
Name: taxa_operacional, dtype: float64

In [92]:
custo_distribuido = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
custo_distribuido['custo'] = 0
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo
0,16956305,2019-10-17,2019-10,BBAS3,False,1000,-4493000,4493000,0
1,16956305,2019-10-17,2019-10,ENEV3,False,1900,-5625900,5625900,0
2,16977078,2019-10-18,2019-10,BBAS3,False,400,-1859200,1859200,0
3,16977078,2019-10-18,2019-10,ITUB4,False,-1400,4809000,4809000,0
4,16977078,2019-10-18,2019-10,QUAL3,False,2000,-6710000,6710000,0
5,16977078,2019-10-18,2019-10,RAPT4,False,7500,-8250000,8250000,0
6,16998228,2019-10-21,2019-10,ITUB4,False,-500,1719500,1719500,0
7,17022090,2019-10-22,2019-10,RAPT4,False,-7500,8252200,8252200,0
8,17045946,2019-10-23,2019-10,ENEV3,False,-1900,5787500,5787500,0
9,17045946,2019-10-23,2019-10,QUAL3,False,-2000,6190000,6190000,0


In [93]:
for nota in set(teste.nota):
    constante = custo_distribuido[custo_distribuido.nota == nota].turnover.sum()
    custo_total = taxas_detalhadas[taxas_detalhadas.nota == nota].custos_operacionais.sum()
    fator = custo_total/constante
    custo_distribuido['custo'] = custo_distribuido['custo']+ custo_distribuido.nota.apply(lambda x : fator if x==nota else 0)*custo_distribuido.turnover

In [94]:
#valor infimo
custo_distribuido.groupby('nota')[['custo']].sum().values - taxas_detalhadas.groupby('nota')[['custos_operacionais']].sum().values

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [95]:
custo_distribuido = custo_distribuido.merge(taxas_detalhadas[['nota','data_liquidacao']],left_on='nota', right_on='nota')
custo_distribuido.head(6)

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao
0,16956305,2019-10-17,2019-10,BBAS3,False,1000,-4493000,4493000,-3414.518377,2019-10-21
1,16956305,2019-10-17,2019-10,ENEV3,False,1900,-5625900,5625900,-4275.481623,2019-10-21
2,16977078,2019-10-18,2019-10,BBAS3,False,400,-1859200,1859200,-1518.172164,2019-10-22
3,16977078,2019-10-18,2019-10,ITUB4,False,-1400,4809000,4809000,-3926.898632,2019-10-22
4,16977078,2019-10-18,2019-10,QUAL3,False,2000,-6710000,6710000,-5479.203540,2019-10-22
5,16977078,2019-10-18,2019-10,RAPT4,False,7500,-8250000,8250000,-6736.725664,2019-10-22


In [96]:
custo_distribuido['aux'] = custo_distribuido['financeiro'] +custo_distribuido['custo']
custo_distribuido['aux']=custo_distribuido['aux'].apply(int)/100
custo_distribuido[['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
0,2019-10-17,2019-10-21,BBAS3,False,-44964.14
1,2019-10-17,2019-10-21,ENEV3,False,-56301.75
2,2019-10-18,2019-10-22,BBAS3,False,-18607.18
3,2019-10-18,2019-10-22,ITUB4,False,48050.73
4,2019-10-18,2019-10-22,QUAL3,False,-67154.79
5,2019-10-18,2019-10-22,RAPT4,False,-82567.36
6,2019-10-21,2019-10-23,ITUB4,False,17167.94
7,2019-10-22,2019-10-24,RAPT4,False,82474.29
8,2019-10-23,2019-10-25,ENEV3,False,57814.86
9,2019-10-23,2019-10-25,QUAL3,False,61835.68


In [97]:
custo_distribuido[custo_distribuido.daytrade].groupby(['pregao','nome']).sum()

,,daytrade,qtd,financeiro,turnover,custo,aux
pregao,nome,,,,,,


In [182]:
(24.09*98700+custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum())*15/100


49052.765999999996

In [173]:
-custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum()

2050664.56

In [174]:
custo_distribuido[~custo_distribuido.daytrade][custo_distribuido.nome!='SQIA3'].aux.sum()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


351459.52

In [165]:
custo_distribuido[custo_distribuido.daytrade].aux.sum()

26152.139999999996

In [104]:
custo_distribuido[custo_distribuido.nome=='oibr3'.upper()][['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
16,2020-05-04,2020-05-06,OIBR3,True,31251.10
35,2020-06-15,2020-06-17,OIBR3,False,-1019030.78
40,2020-06-18,2020-06-22,OIBR3,False,1096598.12
41,2020-06-23,2020-06-25,OIBR3,False,-514059.86
42,2020-06-26,2020-06-30,OIBR3,False,-232093.07
45,2020-07-24,2020-07-28,OIBR3,False,-50437.04
46,2020-07-31,2020-08-04,OIBR3,False,1184096.06


In [101]:
custo_distribuido[custo_distribuido.nome =='BBAS3']

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,16956305,2019-10-17,2019-10,BBAS3,False,1000,-4493000,4493000,-3414.518377,2019-10-21,-44964.14
2,16977078,2019-10-18,2019-10,BBAS3,False,400,-1859200,1859200,-1518.172164,2019-10-22,-18607.18
10,17076198,2019-10-24,2019-10,BBAS3,False,-30200,144420200,144420200,-60106.222752,2019-10-28,1443600.93


In [102]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,16956305,2019-10-17,2019-10,BBAS3,False,1000,-4493000,4493000,-3414.518377,2019-10-21,-44964.14
1,16956305,2019-10-17,2019-10,ENEV3,False,1900,-5625900,5625900,-4275.481623,2019-10-21,-56301.75
2,16977078,2019-10-18,2019-10,BBAS3,False,400,-1859200,1859200,-1518.172164,2019-10-22,-18607.18
3,16977078,2019-10-18,2019-10,ITUB4,False,-1400,4809000,4809000,-3926.898632,2019-10-22,48050.73
4,16977078,2019-10-18,2019-10,QUAL3,False,2000,-6710000,6710000,-5479.203540,2019-10-22,-67154.79
5,16977078,2019-10-18,2019-10,RAPT4,False,7500,-8250000,8250000,-6736.725664,2019-10-22,-82567.36
6,16998228,2019-10-21,2019-10,ITUB4,False,-500,1719500,1719500,-2706.000000,2019-10-23,17167.94
7,17022090,2019-10-22,2019-10,RAPT4,False,-7500,8252200,8252200,-4771.000000,2019-10-24,82474.29
8,17045946,2019-10-23,2019-10,ENEV3,False,-1900,5787500,5787500,-6013.394907,2019-10-25,57814.86
9,17045946,2019-10-23,2019-10,QUAL3,False,-2000,6190000,6190000,-6431.605093,2019-10-25,61835.68


In [127]:
pq

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,16956305,2019-10-17,2019-10,BBAS3,False,1000.0,-4.493000e+06,4.493000e+06,-3414.518377,2019-10-21,-44964.140000
1,16956305,2019-10-17,2019-10,ENEV3,False,1900.0,-5.625900e+06,5.625900e+06,-4275.481623,2019-10-21,-56301.750000
2,16977078,2019-10-18,2019-10,BBAS3,False,400.0,-1.859200e+06,1.859200e+06,-1518.172164,2019-10-22,-18607.180000
3,16977078,2019-10-18,2019-10,ITUB4,False,-1400.0,4.809000e+06,4.809000e+06,-3926.898632,2019-10-22,48050.730000
4,16977078,2019-10-18,2019-10,QUAL3,False,2000.0,-6.710000e+06,6.710000e+06,-5479.203540,2019-10-22,-67154.790000
5,16977078,2019-10-18,2019-10,RAPT4,False,7500.0,-8.250000e+06,8.250000e+06,-6736.725664,2019-10-22,-82567.360000
6,16998228,2019-10-21,2019-10,ITUB4,False,-500.0,1.719500e+06,1.719500e+06,-2706.000000,2019-10-23,17167.940000
7,17022090,2019-10-22,2019-10,RAPT4,False,-7500.0,8.252200e+06,8.252200e+06,-4771.000000,2019-10-24,82474.290000
8,17045946,2019-10-23,2019-10,ENEV3,False,-1900.0,5.787500e+06,5.787500e+06,-6013.394907,2019-10-25,57814.860000
9,17045946,2019-10-23,2019-10,QUAL3,False,-2000.0,6.190000e+06,6.190000e+06,-6431.605093,2019-10-25,61835.680000


In [140]:
pq[pq.nome=='ITUB4'].aux.sum()

-4148.180000000008

In [133]:
5787500.0-5625900.0

161600.0

In [134]:
57814.86-56301.75


1513.1100000000006

In [ ]:
4696.672582781459

In [128]:
custo_distribuido.loc[:11,:].groupby('nome').sum().financeiro.sum()

137306500

In [122]:
custo_distribuido.loc[:11,:].groupby('nome').sum()[['qtd']]

,qtd
nome,
BBAS3,0.0
ENEV3,0.0
ITUB4,0.0
QUAL3,0.0
RAPT4,0.0


In [99]:
custo_distribuido.groupby('nome').sum()[['qtd']]

,qtd
nome,
BBAS3,-28800
BRML3,5000
ENEV3,0
IGTA3,1400
ITUB4,0
MULT3,1300
QUAL3,0
RAPT4,0


In [100]:
qtd = aux.groupby(by=['pregao','nome']).sum()[["qtd"]].reset_index().pivot(index='nome',columns='pregao', values='qtd').fillna(0)
qtd

pregao,2019-10-17,2019-10-18,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-30,2019-10-31
nome,,,,,,,,
BBAS3,1000.0,400.0,0.0,0.0,0.0,-30200.0,0.0,0.0
BRML3,0.0,0.0,0.0,0.0,0.0,0.0,2500.0,2500.0
ENEV3,1900.0,0.0,0.0,0.0,-1900.0,0.0,0.0,0.0
IGTA3,0.0,0.0,0.0,0.0,0.0,0.0,600.0,800.0
ITUB4,0.0,-1400.0,-500.0,0.0,0.0,1900.0,0.0,0.0
MULT3,0.0,0.0,0.0,0.0,0.0,0.0,1300.0,0.0
QUAL3,0.0,2000.0,0.0,0.0,-2000.0,0.0,0.0,0.0
RAPT4,0.0,7500.0,0.0,-7500.0,0.0,0.0,0.0,0.0


In [109]:
financeiro = aux.groupby(by=['pregao','nome']).sum()[["financeiro"]].reset_index().pivot(index='nome',columns='pregao', values='financeiro').fillna(0)
financeiro/100

pregao,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-16,2020-04-24,2020-04-29,2020-04-30,2020-05-04,2020-05-06,...,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-23,2020-06-26,2020-07-02,2020-07-08,2020-07-24,2020-07-31
nome,,,,,,,,,,,,,,,,,,,,,
BOVA11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,702155.8,0.0,-340078.2,-385155.0,0.0,0.0,0.0,0.0,0.0,0.0
BPAC11,0.0,0.0,7700.0,0.0,-606300.0,0.0,0.0,0.0,0.0,0.0,...,61360.0,378852.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNTO3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-101120.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COGN3,0.0,0.0,0.0,0.0,-981000.0,0.0,1006200.0,-221600.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENEV3,3251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBIO33,0.0,-155993.0,-99750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,268731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LWSA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-55253.0,49951.0,0.0,0.0
MGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OIBR3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31746.0,0.0,...,-1018500.0,0.0,0.0,1097000.0,-513880.0,-232000.0,0.0,0.0,-50400.0,1184480.0


In [110]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08,3035.82
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09,-156105.53
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13,7467.83
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13,-99780.17
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13,-345304.41
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13,-1263.05
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,-12393.000000,2020-04-14,335210.07
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,-20889.748868,2020-04-20,-606508.89
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,-33799.841068,2020-04-20,-981337.99
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,-16507.139506,2020-04-20,-479265.07


In [621]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


28407.0

In [623]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-2228.6067927113077

In [635]:
#Imposto de renda relativo a daytrade pra ser pago no final do mes 6,referente ao mês 5
(28407.0-2228.6067927113077-12664.86 -12828.95 )*20/100 + taxas_detalhadas.IRRF_daytrade.sum()/100




-268.4633585422617

In [652]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-109637.65

In [650]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-3976.2850533131127